<a href="https://colab.research.google.com/github/Fahim2280/CVPR/blob/main/Final-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [ ]:
from tensorflow.keras import layers, regularizers

import numpy as np
import matplotlib.pyplot as plt

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def create_and_train_model(optimizer='adam', regularizer=None, epochs=100):
    model = tf.keras.Sequential([
        layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizer, input_shape=(32, 32, 3)),
        layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizer),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizer),
        layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizer),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizer),
        layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizer),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(x=X_train, y=Y_train, epochs=epochs, validation_split=0.2, batch_size=128, callbacks=[early_stop], verbose=0)

    test_loss, test_acc = model.evaluate(X_test, Y_test)
    print(f'\nTest Accuracy (Optimizer: {optimizer}, Regularizer: {regularizer}): {test_acc:.4f}')
    print(f'Test Loss (Optimizer: {optimizer}, Regularizer: {regularizer}): {test_loss:.4f}')

    return model, history

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

epochs = 50

models = []
histories = []

optimizers = ['adam', 'sgd', 'rmsprop']
regularizers = [None, regularizers.l1(0.01), regularizers.l2(0.01)]

for optimizer in optimizers:
    for regularizer in regularizers:
        model, history = create_and_train_model(optimizer=optimizer, regularizer=regularizer, epochs=epochs)
        models.append(model)
        histories.append(history)

plt.figure(figsize=(15, 5))
for i, (optimizer, regularizer) in enumerate(zip(optimizers, regularizers)):
    plt.subplot(1, 3, i + 1)
    plt.plot(histories[i].history['accuracy'], 'o-', label='train accuracy')
    plt.plot(histories[i].history['val_accuracy'], 'o-', label='validation accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.title(f'Optimizer: {optimizer}, Regularizer: {regularizer}')
    plt.legend(loc='lower right')
plt.show()


rows = 5
cols = 5

plt.figure(figsize=(20, 10))
for i in range(rows * cols):
    plt.subplot(rows, 2 * cols, 2 * i + 1)
    plt.imshow(X_test[i])
    plt.xticks([])
    plt.yticks([])
    plt.ylabel(class_names[Y_test[i][0]])
    predicted_label = np.argmax(models[0].predict(np.expand_dims(X_test[i], axis=0)))
    color = 'g' if Y_test[i][0] == predicted_label else 'r'
    plt.xlabel(class_names[predicted_label], color=color)

    plt.subplot(rows, 2 * cols, 2 * i + 2)
    plt.bar(range(10), models[0].predict(np.expand_dims(X_test[i], axis=0))[0])
    plt.yticks([])
    plt.xticks(range(10))
    plt.ylim([0, 1])
    plt.xlim([0, 10])

plt.show()


313/313 [==============================] - 21s 67ms/step - loss: 0.7700 - accuracy: 0.7436

Test Accuracy (Optimizer: adam, Regularizer: None): 0.7436
Test Loss (Optimizer: adam, Regularizer: None): 0.7700
313/313 [==============================] - 22s 72ms/step - loss: 2.6521 - accuracy: 0.1000

Test Accuracy (Optimizer: adam, Regularizer: <keras.regularizers.L1 object at 0x7b16f8dbb040>): 0.1000
Test Loss (Optimizer: adam, Regularizer: <keras.regularizers.L1 object at 0x7b16f8dbb040>): 2.6521


In [ ]:
Conclusion
After examining the search results, we can draw the following conclusions regarding the best approaches to address overfitting and prioritize accuracy in the model.
When dealing with overfitting, L2 regularization combined with the SGD optimizer proves to be the most effective, even with 50 epochs. On the other hand, if the
primary goal is to maximize accuracy, the optimal combination is L2 regularization with the Adam optimizer, even with 64 epochs. Nevertheless, it is worth noting that
accuracy can be further improved without early stopping by using a specific omega value for val_loss with L2 and Adam. Similarly, for L2 and SGD, increasing the number
of epochs can lead to enhanced accuracy. In essence, the choice of optimizer and regularization technique should be based on the specific requirements and objectives
of the model.